## Rename & Repackage Data
The data was in a format I did not prefer so it is renamed and repackaged here. This has to be done in several steps, else there is a memory error.

In [1]:
import pandas as pd
from math import radians, sin, cos

train_path = '../data/train3.pkl'
test_path = '../data/test3.pkl'

def equirectangular_projection(data):
    phi = data.lat.apply(radians)
    lamb = data.lon.apply(radians)
    lamb0 = lamb.mean()
    phi0 = phi.mean()
    r = 6378000  # earth radius in m
    x = r*(phi-phi0)
    y = r*(lamb - lamb0)*cos(phi0)
    return x, y

def compute_velocity(row):
    r = row.speed
    theta = radians(row.direction)
    row['dx'] = r*cos(radians(theta))
    row['dy'] = r*sin(radians(theta))
    return row

def filter_position_observed_events(data):
    return data[data.event == 'ObservedPositionEvent']

def repackage(from_path, to_path):
    data = pd.read_pickle(from_path)
    
    # Remove unnecessary fields
    del data['vehicle_id']
    del data['line']
    del data['station']
        
    data = filter_position_observed_events(data)
    
    del data['event']
    
    data.timestamp = data.timestamp.apply(pd.to_datetime)
    data = data.rename(columns = {
        'latitude': 'lat',
        'longitude': 'lon',
        'journey_number': 'traj',
        'segment_number': 'seg', 
        'speed': 'speed',
        'event': 'event',
        'timestamp': 'timestamp'
    })
    
    x, y = equirectangular_projection(data)
    data['x'] = x
    data['y'] = y
    
    del data['lat']
    del data['lon']

    data = data.apply(compute_velocity, axis=1)
    
    del data['direction']
    del data['speed']
    
    data.to_pickle(to_path)
    
#repackage('../data/203_train.p', train_path)
#repackage('../data/203_test.p', test_path)

In [ ]:
# i think this is shit
#data = pd.read_pickle('../data/train3.pkl')
#data.head()
def mean_timestamp(timestamps):    
    return pd.to_datetime(timestamps.values.astype(np.int64).mean())

X = data.iloc[0].copy()
data.iloc[4].timestamp = 6
X.timestamp = mean_timestamp(data.iloc[:10].timestamp)
compressed_data = data.drop(['timestamp', 'event', 'seg', 'station', 'line', 'traj'], axis=1).apply(np.mean, axis=0)
compressed_data['timestamp'] = mean_timestamp(data.iloc[:10].timestamp)# pd.Timestamp(2017, 1, 1, 12)
#compressed_data.timestamp = mean_timestamp(data.iloc[:10].timestamp)
compressed_data

In [6]:
#data = pd.read_pickle('../data/203_train.p')
data.speed[:100]

0    -1.00
1     0.00
2    -1.00
3     0.00
4     0.00
5     0.00
6     0.00
7     0.00
8     0.00
9     0.00
10    0.00
11    0.00
12    0.00
13    0.00
14    0.00
15    0.00
16    0.00
17   -1.00
18    0.41
19    0.77
20    0.97
21    1.18
22    1.49
23    1.69
24    2.31
25   -1.00
26    2.57
27    2.72
28    2.77
29    2.77
      ... 
70    8.69
71    8.02
72    7.45
73    6.79
74    6.32
75    5.55
76   -1.00
77    4.62
78    4.01
79    3.29
80    2.41
81    1.02
82   -1.00
83    0.00
84    0.00
85    0.00
86    0.00
87    0.00
88    0.00
89    0.00
90    0.00
91    0.00
92    0.00
93    0.00
94   -1.00
95    0.00
96    1.28
97    2.05
98    2.93
99    3.90
Name: speed, Length: 100, dtype: float64